<a href="https://colab.research.google.com/github/Miguelapp10/API_Simpliroute_Urbano/blob/main/API_Urbano.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
# Configurar cliente de BigQuery
import gzip
import shutil
from datetime import datetime
from google.colab import auth
from google.colab import files # Esto solo es necesario si estás usando Google Colab
from google.cloud import bigquery
from google.cloud import storage
import pandas as pd
import os
import gspread as gs
import gspread_dataframe as gd

import concurrent.futures
from google.auth import default

# URL y endpoint para producción
BASE_URL_PROD = "https://api.urbanoexpress.com.pe"
ENDPOINT_PROD = "/api/ws/e-tracking"

# Credenciales de solicitud (reemplazar con tus credenciales reales)
API_KEY = ""

# Suprimir sólo las advertencias de InsecureRequestWarning
warnings.filterwarnings('ignore', category=InsecureRequestWarning)

# Autentificación
auth.authenticate_user()
creds, _ = default()
client = bigquery.Client(credentials=creds, project='')

# Consulta para obtener los números de guía
query = """
"""
numeros_de_guia = client.query(query).to_dataframe()['tracking'].tolist()

# Función para obtener información de seguimiento
def obtener_informacion_tracking(api_key, guia):
    url = f"https://api.urbanoexpress.com.pe/api/ws/e-tracking"
    headers = {
        'x-api-key': api_key,
        'Content-Type': 'application/json'
    }
    data = {
        "guia_ue": guia,
        "tracking_number": ""
    }
    try:
        response = requests.post(url, headers=headers, json=data, verify=False)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
       # print(f"Error en la solicitud para {guia}: {e}")
        return None

# Función para convertir datos a un DataFrame de pandas
def convertir_a_dataframe(informacion):
    if informacion is None:
        return pd.DataFrame()

    # Crear una lista para almacenar las filas de datos
    rows = []

    # Extraer información principal
    info_principal = {

        "guia_ue": informacion.get("guia_ue", ""),
        "tracking_number": informacion.get("tracking_number", ""),

        "servicio_contrato": informacion.get("servicio", {}).get("contrato", ""),
        "servicio_origen": informacion.get("servicio", {}).get("origen", ""),
        "servicio_admitido": informacion.get("servicio", {}).get("admitido", ""),

        "cliente_codigo": informacion.get("cliente", {}).get("codigo", ""),
        "cliente_nombre": informacion.get("cliente", {}).get("nombre", ""),
        "cliente_telefono": informacion.get("cliente", {}).get("telefono", ""),

        "direccion_entrega": informacion.get("direccion_entrega", {}).get("direccion", ""),
        "direccion_entrega_referencia": informacion.get("direccion_entrega", {}).get("referencia", ""),
        "direccion_entrega_ciudad": informacion.get("direccion_entrega", {}).get("ciudad", ""),
        "direccion_entrega_zona": informacion.get("direccion_entrega", {}).get("zona", ""),
        "direccion_entrega_agencia": informacion.get("direccion_entrega", {}).get("agencia", ""),

        "estado_actual_estado": informacion.get("estado_actual", {}).get("estado", ""),
        "estado_actual_Detalle_Estado": informacion.get("estado_actual", {}).get("Detalle_Estado", ""),
        "estado_actual_fecha": informacion.get("estado_actual", {}).get("fecha", "") ,
         "estado_actual_hora":informacion.get("estado_actual", {}).get("hora", "")
    }

    # Extraer información de movimientos
    movimientos = informacion.get("movimiento", [])
    for movimiento in movimientos:
        row = info_principal.copy()  # Copiar información principal
        row.update({
            "movimiento_secuencia": movimiento.get("secuencia", ""),
            "movimiento_codigo": movimiento.get("codigo", ""),
            "movimiento_estado": movimiento.get("estado", ""),
            "movimiento_sub_estado": movimiento.get("sub_estado", ""),
            "movimiento_detalle_estado": movimiento.get("detalle_estado", ""),
            "movimiento_fecha": movimiento.get("fecha", "") + " " + movimiento.get("hora", ""),
            "movimiento_apuntes": movimiento.get("apuntes", ""),
            "movimiento_dir_agencia": movimiento.get("dir_agencia", ""),
            "movimiento_n_visita": movimiento.get("n_visita", "")
        })
        rows.append(row)  # Agregar la fila a la lista

    # Convertir la lista de filas a un DataFrame de pandas
    df = pd.DataFrame(rows)
    # Especificar el formato de las fechas y horas
    date_format = "%Y-%m-%d %H:%M:%S"

    # Convertir la columna de fecha y hora a datetime
    if not df.empty:
        df['movimiento_fecha'] = pd.to_datetime(df['movimiento_fecha'], errors='coerce')
        df['estado_actual_fecha_hora'] = pd.to_datetime(df['estado_actual_fecha'] + " " + df['estado_actual_hora'],  errors='coerce')

        # Eliminar las columnas originales de fecha y hora si ya no son necesarias
        df = df.drop(columns=['estado_actual_fecha', 'estado_actual_hora'])

    return df
# Procesamiento en paralelo usando ThreadPoolExecutor
def procesar_tracking(api_key, numeros_de_guia, max_workers=30):
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        future_to_guia = {executor.submit(obtener_informacion_tracking, api_key, guia): guia for guia in numeros_de_guia}
        resultados = []
        for future in concurrent.futures.as_completed(future_to_guia):
            guia = future_to_guia[future]
            try:
                resultado = future.result()
                if resultado:  # Solo agregar resultados válidos
                    resultados.append(resultado)
            except Exception as e:
                print(f"Error al procesar la información de tracking para {guia}: {e}")
    return resultados

resultados = procesar_tracking(API_KEY, numeros_de_guia, max_workers=30)

# Convertir resultados a DataFrame
df_total = pd.concat([convertir_a_dataframe(r) for r in resultados if r], ignore_index=True)

project_id = ''
!gcloud config set project {project_id}
# Configuración del dataset y la tabla en BigQuery
dataset_id = ''
table_id = 'URBANO_IL'

# Cliente de BigQuery
client = bigquery.Client(project=project_id)
# Cargar el DataFrame en BigQuery
df_total.to_gbq(destination_table=f'{dataset_id}.{table_id}', project_id=project_id, if_exists='replace')

